In [ ]:
# default_exp core

# Library Documentation

> API details for DashComponentBase, DashFigureFactory, DashComponent and DashApp

In [ ]:
#hide
from nbdev.showdoc import *

## imports

In [ ]:
#export

import sys
from abc import ABC
import inspect
import types
from importlib import import_module

import shortuuid
import oyaml as yaml
from urllib.parse import urlparse, parse_qs, urlencode
import ast

import dash
import jupyter_dash

import dash_bootstrap_components as dbc

In [ ]:
#export
import dash_core_components as dcc
import dash_html_components as html

from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate

## DashComponentBase

All classes in the library are derived from `DashComponentBase` which is a an abstract base class (ABC) that provides:

- automatic storing of parameters to attributes
- automatic storing of parameters to a `._stored_params` property
- exporting component to a config dict with `.to_config()`
- exporting component to a yaml file with `.to_yaml()`
- building component from a config dict with classmethod `.from_config()`
- building component from a yaml file with classmethod `.from_yaml()`


In [ ]:
#export
class DashComponentBase(ABC):  
    """Base class for all dash_oop_components classes. 
    
    Stores parameter of child classes to attributes and ._stored_params.
    Proved .to_config(), to_yaml(), .from_config() and .from_yaml() methods
    """
    def __init__(self, no_store=None, no_attr=None, no_config=None, child_depth=3):
        """
        Args:
            no_store {list, bool}: either a list of parameters not to store or True, in which
                case no parameters gets stored.
            no_attr {list, bool}: either a list of parameter not to assign to attribute or True,
                in which case no parameters get assigned to attributes
            no_config {list, bool}: either a list of parameter not to store to ._stored_params
                or True, in which case no parameters get saved to ._stored_params
            child_depth (int): how deep the child is from which parameters will be read.
                Defaults to 3 (i.e. The child of the child of DashComponentBase)
        """
        self._store_child_params(no_store, no_attr, no_config, child_depth)

    def _store_child_params(self, no_store=None, no_attr=None, no_config=None, child_depth=3):
        """
        Args:
            no_store {list, bool}: either a list of parameters not to store or True, in which
                case no parameters gets stored.
            no_attr {list, bool}: either a list of parameter not to assign to attribute or True,
                in which case no parameters get assigned to attributes
            no_config {list, bool}: either a list of parameter not to store to ._stored_params
                or True, in which case no parameters get saved to ._stored_params
            child_depth (int): how deep the child is from which parameters will be read.
                Defaults to 3 (i.e. The child of the child of DashComponentBase)
        """
             
        if not hasattr(self, '_stored_params'): 
            self._stored_params = {}
            
        child_frame = sys._getframe(child_depth)
        child_args = child_frame.f_code.co_varnames[1:child_frame.f_code.co_argcount]
        child_dict = {arg: child_frame.f_locals[arg] for arg in child_args}
        if 'kwargs' in child_frame.f_locals:
            child_dict['kwargs'] = child_frame.f_locals['kwargs']
        
        if isinstance(no_store, bool) and no_store:
            return
        else:
            if no_store is None: no_store = tuple()
        
        if isinstance(no_attr, bool) and no_attr: dont_attr = True
        else:
            if no_attr is None: no_attr = tuple()
            dont_attr = False 
            
        if isinstance(no_config, bool) and no_param: dont_config = True
        else:
            if no_config is None: no_config= tuple()
            dont_config = False 

        for name, value in child_dict.items():
            if name in {'dash_component', 'dash_figure_factory', 'dash_app'}:
                raise ValueError(f"Please do not use {name} as a parameter name, "
                                 "as this results in a confusing and hard to parse config.")
            if not dont_attr and name not in no_store and name not in no_attr:
                setattr(self, name, value)
            if not dont_config and name not in no_store and name not in no_config:
                self._stored_params[name] = value
      
    def to_config(self):
        """
        returns a dict with class name, module and params 
        """
        return dict(dash_component=dict(
            class_name=self.__class__.__name__, 
            module=self.__class__.__module__,
            params=self._stored_params))
        
    def to_yaml(self, filepath=None):
        """
        stores a yaml configuration to disk. 
        
        If no filepath is given, returns a str of the yaml config.
        """
        yaml_config = self.to_config()
        if filepath is not None:
            yaml.dump(yaml_config, open(filepath, "w"))
            return
        return yaml.dump(yaml_config)
    
    def dump(self, filepath=None):
        """store the object to disk. 
        
        Default serializer is pickle, however depending on file suffix, 
        dill or joblib will be used."""
        if filepath is None and hasattr(self, "filepath"):
            filepath = self.filepath
        filepath = str(filepath)
        if filepath.endswith(".pkl") or filepath.endswith(".pickle"):
            import pickle
            pickle.dump(self, open(filepath, "wb"))
        elif filepath.endswith(".dill"):
            import dill
            dill.dump(self, open(filepath, "wb"))
        elif str(filepath).endswith(".joblib"):
            import joblib
            joblib.dump(self, filepath)
        else:
            filepath = filepath + ".pkl"
            import pickle
            pickle.dump(self, open(filepath, "wb"))
    
    @classmethod
    def from_config(cls, config, try_pickles=False, force_pickles=False, **update_params):
        """
        Loads a dash_oop_component class from a configuration dict.
        
        Args:
            config (dict): configuration dict, generated from .to_config()
            try_pickles (bool): when finding a filepath parameter try loading from that file,
                if it fails, load from config
            force_pickles (bool): when finding a filepath parameter try loading from that file,
                if it fails, raise error
            **update_params: kwargs that override settings in params
        
        Returns:
            Instance of the class defined in the config.
        """
        if 'dash_component' in config:
            config = config['dash_component']
        elif 'dash_figure_factory' in config:
            config = config['dash_figure_factory']
        elif 'dash_app' in config:
            config = config['dash_app']
        else:
            raise ValueError("I only know how to build dash_component, "
                             "dash_figure_factory and dash_app from config!", str(config))
        
        
        
        params = config['params']
        if not params: params = {}
            
        if (try_pickles or force_pickles) and 'filepath' in params and params['filepath'] is not None:
            try:
                filepath = params['filepath']
                return cls.from_file(filepath)
            except FileNotFoundError as e:
                if force_pickles:
                    raise FileNotFoundError(
                        f"Couldn't find {filepath}! Either change the filepath"
                        " in the configuration or the yaml file, pass "
                        "force_filepath=False or try_filepath=True, or pass the "
                        "correct filepath as **kwargs...")
                else:
                    print(f"Couldn't find {filepath}! You could try passing correct filepath as kwarg."
                          "Now loading from config instead...", flush=True)
            
        for k, v in update_params.items():
            if k in params:
                params[k] = v
            elif 'kwargs' in params:
                params['kwargs'][k]=v
            else:
                raise ValueError(f"This dash_oop_component does not take {k} as an argument, "
                                "nor does it take **kwargs!")
        
        for k, v in params.items():
            if isinstance(v, dict) and len(v)==1 and ('dash_figure_factory' in v or 'dash_component' in v):
                if try_pickles or force_pickles:
                    if ('dash_component' in v 
                        and 'filepath' in v['dash_component']['params']):
                        filepath = v['dash_component']['params']['filepath']
                    elif ('dash_figure_factory' in v 
                          and 'filepath' in v['dash_figure_factory']['params']):
                        filepath = v['dash_figure_factory']['params']['filepath']
                    else:
                        filepath = None
                    if filepath is not None:
                        try:
                            params[k] = DashComponentBase.from_file(filepath)
                        except FileNotFoundError as e:
                            if force_pickles:
                                raise FileNotFoundError(
                                    f"Couldn't find {filepath}! Either change the filepath"
                                    " in the configuration or the yaml file, pass "
                                    "force_filepath=False or pass try_filepath=True")
                            else:
                                print(f"Couldn't find {filepath}! So loading from config instead...", flush=True)
                                params[k] = DashComponentBase.from_config(v)             
                    else:
                        params[k] = DashComponentBase.from_config(v)
                else:
                    params[k] = DashComponentBase.from_config(v)
        
        try:
            component_class = getattr(import_module(config['module']), config['class_name'])
        except ModuleNotFoundError:
            import sys
            from pathlib import Path
            sys.path.append(str(Path.cwd()))
            component_class = getattr(import_module(config['module']), config['class_name'])
            
        if 'kwargs' in params:
            kwargs = params.pop('kwargs')
        else:
            kwargs = {}
        
        if "name" in params:
            name = params.pop('name')
        else:
            name = None
            
        comp = component_class(**params, **kwargs)
        if name is not None:
            comp.name = name
        return comp
    
    @classmethod
    def from_yaml(cls, yaml_filepath, try_pickles=False, force_pickles=False,  **update_params):
        """
        Loads a dash_oop_component class from a yaml file.
        
        Args:
            yaml_filepath (str, Path): filepath of a .yaml file, generated from .to_yaml()
            try_pickles (bool): when finding a filepath parameter try loading from that file,
                if it fails, load from config
            force_pickles (bool): when finding a filepath parameter try loading from that file,
                if it fails, raise errors
            update_params: a dict of parameters to be overridden by update_params
        
        Returns:
            Instance of the class defined in the yaml file.
        """
        config = yaml.safe_load(open(str(yaml_filepath), "r"))
        return cls.from_config(config, try_pickles, force_pickles, **update_params)
    
    @classmethod
    def from_file(cls, filepath):
        """Load a DashComponentBase from file. Depending on the suffix of the filepath 
        will either load with pickle ('.pkl'), dill ('.dill') or joblib ('joblib').
        
        If no suffix given, will try with pickle (and try adding ''.pkl')
        
        Args:
            filepath {str, Path} the location of the stored component
            
        returns:
            DashComponentBase
        """
        filepath = str(filepath)
        if filepath.endswith(".pkl") or str(filepath).endswith(".pickle"):
            import pickle
            return pickle.load(open(filepath, "rb"))
        elif filepath.endswith(".dill"):
            import dill
            return dill.load(open(filepath, "rb"))
        elif filepath.endswith(".joblib"):
            import joblib
            return joblib.load(filepath)
        else:
            from pathlib import Path
            filepath = Path(filepath)
            if not filepath.exists(): 
                if (filepath.parent / (filepath.name + ".pkl")).exists():
                    filepath = filepath.parent / (filepath.name + ".pkl")
                else:
                    raise ValueError(f"Cannot find file: {str(filepath)}")
            import pickle
            return pickle.load(open(str(filepath), "rb"))
        

In [ ]:
show_doc(DashComponentBase.to_config)
show_doc(DashComponentBase.to_yaml)
show_doc(DashComponentBase.from_config)
show_doc(DashComponentBase.from_config)

<h4 id="DashComponentBase.to_config" class="doc_header"><code>DashComponentBase.to_config</code><a href="__main__.py#L68" class="source_link" style="float:right">[source]</a></h4>

> <code>DashComponentBase.to_config</code>()

returns a dict with class name, module and params 

<h4 id="DashComponentBase.to_yaml" class="doc_header"><code>DashComponentBase.to_yaml</code><a href="__main__.py#L77" class="source_link" style="float:right">[source]</a></h4>

> <code>DashComponentBase.to_yaml</code>(**`filepath`**=*`None`*)

stores a yaml configuration to disk. 

If no filepath is given, returns a str of the yaml config.

<h4 id="DashComponentBase.from_config" class="doc_header"><code>DashComponentBase.from_config</code><a href="__main__.py#L111" class="source_link" style="float:right">[source]</a></h4>

> <code>DashComponentBase.from_config</code>(**`config`**, **`try_pickles`**=*`False`*, **`force_pickles`**=*`False`*, **\*\*`update_params`**)

Loads a dash_oop_component class from a configuration dict.

Args:
    config (dict): configuration dict, generated from .to_config()
    try_pickles (bool): when finding a filepath parameter try loading from that file,
        if it fails, load from config
    force_pickles (bool): when finding a filepath parameter try loading from that file,
        if it fails, raise error
    **update_params: kwargs that override settings in params

Returns:
    Instance of the class defined in the config.

<h4 id="DashComponentBase.from_config" class="doc_header"><code>DashComponentBase.from_config</code><a href="__main__.py#L111" class="source_link" style="float:right">[source]</a></h4>

> <code>DashComponentBase.from_config</code>(**`config`**, **`try_pickles`**=*`False`*, **`force_pickles`**=*`False`*, **\*\*`update_params`**)

Loads a dash_oop_component class from a configuration dict.

Args:
    config (dict): configuration dict, generated from .to_config()
    try_pickles (bool): when finding a filepath parameter try loading from that file,
        if it fails, load from config
    force_pickles (bool): when finding a filepath parameter try loading from that file,
        if it fails, raise error
    **update_params: kwargs that override settings in params

Returns:
    Instance of the class defined in the config.

### Example use of DashComponentBase

Define a class `T` as a childclass of `DashComponentBase` and initialize an instance `t`:

In [ ]:
class T(DashComponentBase): 
    def __init__(self, a=1, b=2, **kwargs):
        super().__init__(child_depth=2)
        
t = T(a=2, b=3)

Get the configuration of the instance `t`:

In [ ]:
print(t.to_config())

{'dash_component': {'class_name': 'T', 'module': '__main__', 'params': {'a': 2, 'b': 3, 'kwargs': {}}}}


Get the configuration in `yaml` format:

In [ ]:
print(t.to_yaml())

dash_component:
  class_name: T
  module: __main__
  params:
    a: 2
    b: 3
    kwargs: {}



Tests showing parameters have been assigned to attributes and  config stores all relevenat data to rebuild the instance from the config:
- parameters have been assigned to attributes
- class_name and module have been recorded
- parameters have been recorded

In [ ]:
assert t.a == 2
assert t.b == 3
assert t.to_config()['dash_component']['class_name'] == "T"
assert t.to_config()['dash_component']['module'] == "__main__"
assert t.to_config()['dash_component']['params']['a'] == t.a
assert t.to_config()['dash_component']['params']['b'] == t.b

Store to yaml:

In [ ]:
t.to_yaml("T.yaml")

Load from yaml and check that loaded instance `t2` is same as `t`:

In [ ]:
t2 = T.from_yaml("T.yaml")

assert t2.to_config()['dash_component']['class_name'] == "T"
assert t2.to_config()['dash_component']['module'] == "__main__"
assert t2.to_config()['dash_component']['params']['a'] == t.a
assert t2.to_config()['dash_component']['params']['b'] == t.b

Override parameters upon loading:

In [ ]:
t2 = T.from_yaml("T.yaml", b=4)
assert t2.b == 4

If `**kwargs` in the class definition than unknown parameters get loads to `self.kwargs`:

In [ ]:
t2 = T.from_yaml("T.yaml", c=5)
assert hasattr(t2, "kwargs")
assert "c" in t2.kwargs
assert t2.kwargs["c"] == 5

Store to file ("pickle", although also works with dill and joblib) and reload from file:

In [ ]:
t2.dump("t") # stores to "t.pkl"

In [ ]:
t3 = DashComponentBase.from_file("t.pkl")
print(t3.to_yaml())
assert t3.kwargs["c"] == 5

dash_component:
  class_name: T
  module: __main__
  params:
    a: 2
    b: 3
    kwargs:
      c: 5



If suffix is ".pkl" can simply leave it out. And you can load from any `DashComponentBase` child class:

In [ ]:
t4 =  T.from_file("t")
print(t4.to_yaml())
assert t4.kwargs["c"] == 5

dash_component:
  class_name: T
  module: __main__
  params:
    a: 2
    b: 3
    kwargs:
      c: 5



## DashFigureFactory

A `DashFigureFactory` loads data and provides plots, tables, lists, dicts, etc that can be used for building your dashboard. 

This provides a clean seperation where all your data preparation and visualisation logic goes into one place (the `DashFigureFactory`) and all the dashboard layout and interaction logic goes into another (the `DashComponents`). This means that you only have to change the data representation in one place, and the whole dashboard will adapt.

When used as a parameter to a `DashComponent` (e.g. parameter `figure_factory`)the parameter `figure_factory` gets replaced by the figure factory configuration dict. This means that a `DashComponent` automatically loads its underlying `DashFigureFactory` parameters upon load! 

In some cases you may perform a lot of costly calculations during the initialization of the `DashFigureFactory` (e.g. calculate SHAP values). In this case you may want to pickle the `DashFigureFactory` and reload it from file. If you add a parameter `filepath` to the parameters and `.dump()` the `DashFigureFactory`, then you can automatically reload it from pickle by adding `.from_yaml("component.yaml", try_pickles=True)`.

In [ ]:
#export

class DashFigureFactory(DashComponentBase):
    """
    Helper class to store data for a dashboard and provide e.g. plotting functions.
    
    You should seperate the datastorage/plotting logic from the dashboard logic.
    All data/plotting logic goes into a DashFigureFactory.
    
    All dashboard logic goes into a DashComponent.
    
    Stores to config under key 'dash_figure_factory'
    """
    def __init__(self, no_store=None, no_attr=None, no_config=None):
        super().__init__(no_store=None, no_attr=None, no_config=None)
        
    def to_config(self):
        return dict(dash_figure_factory=dict(
            class_name=self.__class__.__name__, 
            module=self.__class__.__module__,
            params=self._stored_params))
    

In [ ]:
show_doc(DashFigureFactory)

<h2 id="DashFigureFactory" class="doc_header"><code>class</code> <code>DashFigureFactory</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>DashFigureFactory</code>(**`no_store`**=*`None`*, **`no_attr`**=*`None`*, **`no_config`**=*`None`*) :: [`DashComponentBase`](/dash_oop_components/core.html#DashComponentBase)

Helper class to store data for a dashboard and provide e.g. plotting functions.

You should seperate the datastorage/plotting logic from the dashboard logic.
All data/plotting logic goes into a DashFigureFactory.

All dashboard logic goes into a DashComponent.

Stores to config under key 'dash_figure_factory'

### Example use of `DashFigureFactory`: `ListFactory`

A DashFigureFactory that simply stores a list and has a `return_list()` method:

In [ ]:
class ListFactory(DashFigureFactory):
    def __init__(self, list_input, filepath=None):
        super().__init__()
        
    def list_length(self):
        return len(self.list_input)
        
    def return_list(self, first_n_items=None):
        if first_n_items is None: first_n_items = self.list_length()
        if first_n_items <= len(self.list_input):
            return self.list_input[:first_n_items]
        return None

In [ ]:
list_factory = ListFactory(["this", "is", "a", "dumb", "example"], filepath="list_factory.pkl")
print(list_factory.return_list())

['this', 'is', 'a', 'dumb', 'example']


In [ ]:
print(list_factory.to_yaml())

dash_figure_factory:
  class_name: ListFactory
  module: __main__
  params:
    list_input:
    - this
    - is
    - a
    - dumb
    - example
    filepath: list_factory.pkl



In [ ]:
assert list_factory.list_input == ['this', 'is', 'a', 'dumb', 'example']
assert list_factory.return_list() == ['this', 'is', 'a', 'dumb', 'example']
assert list_factory.return_list(2) == ['this', 'is']

Storing and loading from yaml:

In [ ]:
list_factory.to_yaml("list_factory.yaml")

In [ ]:
list_factory2 = DashFigureFactory.from_yaml("list_factory.yaml")

assert list_factory2.list_input == list_factory.list_input
assert list_factory2.return_list() == list_factory.return_list()
assert list_factory2.return_list(2) == list_factory.return_list(2)

- If you use `force_pickles=True` but filepath does not exist, then it will throw a FileNotFoundError.
- If you use `try_pickles=True` but filepath does not exist, then it will simply show a warning.

In [ ]:
list_factory = ListFactory(["this", "is", "a", "dumb", "example"], filepath="list_factory2.pkl")
list_factory.to_yaml("list_factory2.yaml")
try:
    ListFactory.from_yaml("list_factory2.yaml", force_pickles=True)
except Exception as e:
    assert isinstance(e, FileNotFoundError)
list_factory = ListFactory.from_yaml("list_factory2.yaml", try_pickles=True)

Couldn't find list_factory2.pkl! You could try passing correct filepath as kwarg.Now loading from config instead...


So you have to make sure the file exists by dumping it:

In [ ]:
list_factory = ListFactory(["this", "is", "a", "dumb", "example"], filepath="list_factory.pkl")
list_factory.to_yaml("list_factory.yaml")
list_factory.dump()
list_factory2 = ListFactory.from_yaml("list_factory.yaml", force_pickles=True)
assert isinstance(list_factory2, ListFactory)

## `DashComponent`

A `DashComponent` combines a `dash` layout with `dash` callbacks. 

It provides:
- a `.layout()` method that returns the layout for the component
- a `.register_callbacks(app)` method that allow you to register the 
    callbacks of the component to a specific dash app
- automatic conversion of any `DashFigureFactory` parameters to it's config 
    (this means that figure factory will be automatically loaded upon load)
- automatic registration of all `DashComponent` subcomponents in its attributes
    (which means that callbacks of all subcomponents will also automatically be registered)
- a `make_hideable` staticmethod that makes it easy to hide parts of a layout 
    depending on configuration bools
- a `querystring()` method that allows you to store the state of your dashboard in the url querystring,
    and then load the state of the dashboard back from that url.
    
It inherits from `DashComponentBase` so:
- All parameters are automatically stored to attributes and to `._stored_params`
- Can be exported and loaded from config and yaml
- Can be dumped to pickle/dill/joblib and loaded `.from_file()`

**IMPORTANT**: 

- add `+self.name` to all the `id`'s in your layout to make sure layout `id`'s are unique.
- define your callbacks in `_register_callbacks(self, app)` (**note the underscore!**)
- If you're using `self.querystring(params)(...)`, then set the `self.name` of the component 
    to something definitive and readable, as otherwise each run it gets assigned a new random 
    uuid string everytime you reboot your app, breaking previously generated querystring urls.


In [ ]:
#export

class DashComponent(DashComponentBase):
    """DashComponent is a bundle of a dash layout and callbacks that
    can make use of DashFigureFactory objects. 

    A DashComponent can have DashComponent subcomponents, that
    you register with register_components(). 

    DashComponents allow you to:

    1. Write clean, re-usable, composable code for your dashboard
    2. Store your dashboard to config files
    3. Load your dashboard from config files

    Each DashComponent should have a unique .name so that dash id's don't clash.
    If no name is given, DashComponent generates a unique uuid name. This allows
    for multiple instance of the same component type in a single layout. 
    But remember to add `+self.name` to all id's.

    Important:
        define your callbacks in `_register_callbacks()` (note underscore!) and
        DashComponent will register callbacks of subcomponents in addition
        to _register_callbacks() when calling register_callbacks()
    """
    def __init__(self, title="Dash", name=None, 
                 no_store=None, no_attr=None, no_config=None):
        """initialize the DashComponent

        Args:
            title (str, optional): Title of component. Defaults to "Dash".
            name (str, optional): unique name to add to Component elements. 
                        If None then random uuid is generated to make sure 
                        it's unique. Defaults to None.
        """
        super().__init__(no_store, no_attr, no_config)
        self._convert_ff_config_params()
        
        self.title = title
        if not hasattr(self, "name"):
            self.name = name
        if self.name is None:
            self._generate_uuid_name()
        self._stored_params["name"] = self.name
        
        self._components = []
        self.compute_querystring_params(whole_tree=False)
        
    
    
    def _generate_uuid_name(self):
        self.name = str(shortuuid.ShortUUID().random(length=10))

        
    def _convert_ff_config_params(self):
        """convert any DashFigureFactory in the ._stored_params dict to its config"""
        for k, v in self._stored_params.items():
            if isinstance(v, DashFigureFactory):
                self._stored_params[k] = self._stored_params[k].to_config()
                
    @staticmethod
    def make_hideable(element, hide=False):
        """helper function to optionally not display an element in a layout.        

        Example:
            make_hideable(dbc.Col([cutoff.layout()]), hide=hide_cutoff)

        Args:
            hide(bool): wrap the element inside a hidden html.div. If the element 
                        is a dbc.Col or a dbc.FormGroup, wrap element.children in
                        a hidden html.Div instead. Defaults to False.
        """ 
        if hide:
            if isinstance(element, dbc.Col) or isinstance(element, dbc.FormGroup):
                return html.Div(element.children, style=dict(display="none"))
            else:
                return html.Div(element, style=dict(display="none"))
        else:
            return element
        
    def querystring(self, params, *attrs):
        """
        wrapper function that applies params loaded from querystrings
        to the underlying dash layout function's attributes attrs. By 
        default it only applies to the "value" attribute.
          
        Use:
            To only track the value attribute:
                self.querystring(params)(dcc.Input)(id="input-"+self.name, value=1)
            
            To track specific attribute(s):
            
                self.querystring(params, "value", "min", "max")(dcc.Slider)(id="input-"+self.name)
            
        if params=='_store_querystring_params':
            stores a list of tuple(id, attribute) to be tracked in the querystring
            to self._querystring_params. All (nested) querystring parameters can 
            be accessed with .get_querystring_params()
        
        """
        def move_value_to_front(attrs):
            """the value attribute gets encoded with a single querystring, e.g. '?param=1' 
                    gets parsed as param.value=1.
                All other attributes get encoded with two querystrings,  e.g. 
                    '?param=max&param=10' gets parsed as param.max=10.
                Therefore an uneven number of attributes implies that the value attribute
                has been encoded. By always putting value first, we can simply take the
                first querystring param as value, and then parse the rest."""
            if not attrs:
                # if not attributes passed: encode value attribute
                return tuple(['value'])
            if not 'value' in attrs:
                # if 'value' not in attributes then can keep the order as passed
                return attrs
            else:
                # if 'value' in attributes, then make sure it's first in the list:
                attrs_list = list(attrs)
                attrs_list.insert(0, attrs_list.pop(attrs_list.index('value')))
                return tuple(attrs_list)
        
        if params=="_store_querystring_params":
            attrs = move_value_to_front(attrs)
            def wrapper(func):
                def apply_value(*args, **kwargs):
                    for attr in attrs:
                        self._querystring_params.append(tuple([kwargs['id'], attr]))
                    return func(*args, **kwargs)
                return apply_value
            return wrapper
        if params is None:
            def wrapper(func):
                def apply_value(*args, **kwargs):
                    return func(*args, **kwargs)
                return apply_value
            return wrapper
        else:
            def wrapper(func):
                def apply_value(*args, **kwargs):
                    if 'id' in kwargs and kwargs['id'] in params:
                        param_values = params[kwargs['id']]
                        for pv in param_values:
                            kwargs[pv[0]] = pv[1]
                    return func(*args, **kwargs)
                return apply_value
            return wrapper
        
    def get_querystring_params(self):
        """
        Returns a list of tuple(id, attribute) of all element attributes
        in all (sub-)components that have been wrapped by self.querystring() 
        and should be tracked in the url querystring."""
        
        _params = []
        _params.extend(self._querystring_params)
        
        self.register_components()
        for comp in self._components:
            _params.extend(comp.get_querystring_params())
        return _params
    
    def _clear_querystring_params(self, whole_tree=True):
        """clears the querystring params. 
        
        Args:
            whole_tree (bool): if True, clear all _querystring_prams\
                in all subcomponents."""
        self._querystring_params = []
        
        if whole_tree:
            self.register_components()
            for comp in self._components:
                comp._clear_querystring_params(whole_tree)
    
    def compute_querystring_params(self, whole_tree=True):
        """compute ._querystring_params. 
        
        Args:
            whole_tree (bool): if True, compute all _querystring_prams\
                in all subcomponents."""
        self._querystring_params = []
        
        try:
            self.layout("_store_querystring_params")
        except:
            pass
        
        if whole_tree:
            self.register_components()
            for comp in self._components:
                comp.compute_querystring_params(whole_tree)
                
    def get_unreachable_querystring_params(self):
        """returns all element (id, attr) querystring parameters 
        that have a self.querystring() wrapper but because params
        has not been passed correctly down to subcomponents .layout()
        function, will not actually get updated.
        """
        try:
            _ = self.layout(None)
        except:
            return self.get_querystring_params()
    
        self._clear_querystring_params(whole_tree=True)
        _ = self.layout("_store_querystring_params")
        reachable_params = self.get_querystring_params()
        
        self.compute_querystring_params(whole_tree=True)
        all_params = self.get_querystring_params()
        
        unreachable_params = [param for param in all_params if param not in reachable_params]
        return unreachable_params
    
    def id(self, component_id):
        if self.name is None:
            self._generate_uuid_name()
        return component_id + '-' + self.name
        
    def Input(self, component_id, component_property):
        if self.name is None:
            self._generate_uuid_name()
        return Input(component_id + '-' + self.name, component_property)
    
    def Output(self, component_id, component_property):
        if self.name is None:
            self._generate_uuid_name()
        return Output(component_id + '-' + self.name, component_property)
    
    def State(self, component_id, component_property):
        if self.name is None:
            self._generate_uuid_name()
        return State(component_id + '-' + self.name, component_property)
        
    def register_components(self): 
        """register subcomponents so that their callbacks will be registered
        
        Searches self.__dict__, finds all DashComponents and adds them to self._components
        """
        if not hasattr(self, '_components'):
            self._components = []
        for k, v in self.__dict__.items():
            if k != '_components' and isinstance(v, DashComponent) and v not in self._components:
                self._components.append(v)
                
    def tabs(self, params, id, tabs):
        return dcc.Tabs(id=id, value=id+"-"+tabs[0].name, 
                     children=[
                         dcc.Tab(id=id+"-"+tab.name, 
                                 value=id+"-"+tab.name, 
                                 label=tab.title, 
                                 children=tab.layout(params)) for tab in tabs])
    
    def layout(self, params=None):
        """layout to be defined by the particular ExplainerComponent instance.
        All element id's should append +self.name to make sure they are unique."""
        return None
    
    def component_callbacks(self, app):
        """register callbacks specific to this ExplainerComponent."""
        if hasattr(self, "_register_callbacks"):
            print("Warning: the use of _register_callbacks() will be deprecated!"
                  " Use component_callbacks() from now on.")
            self._register_callbacks(app)

    def register_callbacks(self, app):
        """First register callbacks of all subcomponents, then call
        _register_callbacks(app)
        """
        self.register_components()
        for comp in self._components:
            comp.register_callbacks(app)
        self.component_callbacks(app)

In [ ]:
show_doc(DashComponent)
show_doc(DashComponent.make_hideable)
show_doc(DashComponent.querystring)
show_doc(DashComponent.get_querystring_params)
show_doc(DashComponent.get_unreachable_querystring_params)
show_doc(DashComponent.layout)
show_doc(DashComponent.component_callbacks)
show_doc(DashComponent.register_callbacks)


<h2 id="DashComponent" class="doc_header"><code>class</code> <code>DashComponent</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>DashComponent</code>(**`title`**=*`'Dash'`*, **`name`**=*`None`*, **`no_store`**=*`None`*, **`no_attr`**=*`None`*, **`no_config`**=*`None`*) :: [`DashComponentBase`](/dash_oop_components/core.html#DashComponentBase)

DashComponent is a bundle of a dash layout and callbacks that
can make use of DashFigureFactory objects. 

A DashComponent can have DashComponent subcomponents, that
you register with register_components(). 

DashComponents allow you to:

1. Write clean, re-usable, composable code for your dashboard
2. Store your dashboard to config files
3. Load your dashboard from config files

Each DashComponent should have a unique .name so that dash id's don't clash.
If no name is given, DashComponent generates a unique uuid name. This allows
for multiple instance of the same component type in a single layout. 
But remember to add `+self.name` to all id's.

Important:
    define your callbacks in `_register_callbacks()` (note underscore!) and
    DashComponent will register callbacks of subcomponents in addition
    to _register_callbacks() when calling register_callbacks()

<h4 id="DashComponent.make_hideable" class="doc_header"><code>DashComponent.make_hideable</code><a href="__main__.py#L61" class="source_link" style="float:right">[source]</a></h4>

> <code>DashComponent.make_hideable</code>(**`element`**, **`hide`**=*`False`*)

helper function to optionally not display an element in a layout.        

Example:
    make_hideable(dbc.Col([cutoff.layout()]), hide=hide_cutoff)

Args:
    hide(bool): wrap the element inside a hidden html.div. If the element 
                is a dbc.Col or a dbc.FormGroup, wrap element.children in
                a hidden html.Div instead. Defaults to False.

<h4 id="DashComponent.querystring" class="doc_header"><code>DashComponent.querystring</code><a href="__main__.py#L81" class="source_link" style="float:right">[source]</a></h4>

> <code>DashComponent.querystring</code>(**`params`**, **\*`attrs`**)

wrapper function that applies params loaded from querystrings
to the underlying dash layout function's attributes attrs. By 
default it only applies to the "value" attribute.
  
Use:
    To only track the value attribute:
        self.querystring(params)(dcc.Input)(id="input-"+self.name, value=1)
    
    To track specific attribute(s):
    
        self.querystring(params, "value", "min", "max")(dcc.Slider)(id="input-"+self.name)
    
if params=='_store_querystring_params':
    stores a list of tuple(id, attribute) to be tracked in the querystring
    to self._querystring_params. All (nested) querystring parameters can 
    be accessed with .get_querystring_params()

<h4 id="DashComponent.get_querystring_params" class="doc_header"><code>DashComponent.get_querystring_params</code><a href="__main__.py#L147" class="source_link" style="float:right">[source]</a></h4>

> <code>DashComponent.get_querystring_params</code>()

Returns a list of tuple(id, attribute) of all element attributes
in all (sub-)components that have been wrapped by self.querystring() 
and should be tracked in the url querystring.

<h4 id="DashComponent.get_unreachable_querystring_params" class="doc_header"><code>DashComponent.get_unreachable_querystring_params</code><a href="__main__.py#L192" class="source_link" style="float:right">[source]</a></h4>

> <code>DashComponent.get_unreachable_querystring_params</code>()

returns all element (id, attr) querystring parameters 
that have a self.querystring() wrapper but because params
has not been passed correctly down to subcomponents .layout()
function, will not actually get updated.

<h4 id="DashComponent.layout" class="doc_header"><code>DashComponent.layout</code><a href="__main__.py#L252" class="source_link" style="float:right">[source]</a></h4>

> <code>DashComponent.layout</code>(**`params`**=*`None`*)

layout to be defined by the particular ExplainerComponent instance.
All element id's should append +self.name to make sure they are unique.

<h4 id="DashComponent.component_callbacks" class="doc_header"><code>DashComponent.component_callbacks</code><a href="__main__.py#L263" class="source_link" style="float:right">[source]</a></h4>

> <code>DashComponent.component_callbacks</code>(**`app`**)

register callbacks specific to this ExplainerComponent.

<h4 id="DashComponent.register_callbacks" class="doc_header"><code>DashComponent.register_callbacks</code><a href="__main__.py#L270" class="source_link" style="float:right">[source]</a></h4>

> <code>DashComponent.register_callbacks</code>(**`app`**)

First register callbacks of all subcomponents, then call
_register_callbacks(app)

### `DashComponentTabs`

Tracking every single parameter in a big multi tab dashboard can result in extremely long querystring urls. 
However often you would only want to share the analysis on a single tab anyway, so we can shorten the url by only keeping track of the parameters of the tab that you are on at the moment. To make this possible we define a drop-in replacement for `dcc.Tabs` called `DashComponentTabs`.

When you pass an `id` and a list of `DashComponents`, this generates automatically
a
```
dcc.Tabs(id=id, children = [dcc.Tab(tab.layout(), label=tab.title) for tab in tabs])
```

When you pass a `component` parameter and do not set `single_tab_querystrings=False`, then `DashComponentTabs` stores the querystring parameter for each tab in `component._tab_params`. This will get used by `DashApp` to exclude all parameters that are not on the current tab from being stored in the querystring url.

Example usage:

```python
    self.querystring(params)(DashComponentTabs)(
                component=self, id="tabs", tabs=[self.list1, self.list2], params=params)
```



In [ ]:
#export
class DashComponentTabs(dcc.Tabs):
    
    def __init__(self, id=None, tabs=None, params=None, value=None, 
                 component=None, single_tab_querystrings=True, **kwargs):
        assert id is not None, "Need to pass an id! id cannot be None!"
        assert tabs is not None, "Need to pass a list of tabs!"
        assert all([isinstance(tab, DashComponent) for tab in tabs]), \
            "all items in tabs should be a DashComponent!"
        assert len(set([tab.name for tab in tabs]))==len(tabs), \
            "all tabs should have a unique .name property!"
        
        children = [dcc.Tab(id=id+"-"+tab.name, 
                         value=tab.name, 
                         label=tab.title, 
                         children=tab.layout(params)) for tab in tabs]
        if value is None:
            value = tabs[0].name # default to first tab
        if "value" in kwargs:
            del kwargs["value"]
        if component is not None and single_tab_querystrings:
            if not hasattr(component, "_tab_params"):
                component._tab_params = {}
            
            component._tab_params[id] = {
                tab.name: tab.get_querystring_params() for tab in tabs}
            
        super().__init__(id=id, children=children, value=str(value), **kwargs)

### `DashConnector`

In [ ]:
#export

class DashConnector(DashComponent):
        
    def register_components(self): 
        """register subcomponents so that their callbacks will be registered.
        
        In the case of a DashConnector this is a dummy, as no subcomponents need to
        be registered.
        """
        if not hasattr(self, '_components'):
            self._components = []  

### Example use of `DashComponent`:


In [ ]:
import dash_html_components as html
import dash_core_components as dcc

#### Single DashComponent

- You can pass a `name` to a `DashComponent` to be used as a unique suffix for all the layout id's.
    If you don't pass a name, a random uuid identifier `self.name` gets generated instead. However if 
    you are going to track querystrings, it's best to pass a specific, short and readable name.
- You can store the state of your dashboard to the url querystring by wrapping the
    elements that you would like to track in a `self.querystrings(params)(...)` wrapper. 
    - The state of the querystring will be passed down to the layout function as `params`.
    - You pass these params to `self.querystring`, and indicate which elements of the layour function
        you would like to track and reload from state.
    - If no attributes are listed, by default "value" gets tracked
- callbacks should be defined in `_register_callbacks(app)` (**note the underscore!**)
    - the `register_callbacks(app)` will then first register all callbacks 
        of subcomponents and then call this `_register_callbacks(app)` method.
 

In [ ]:
class T(DashComponent):
    def __init__(self, name=None):
        super().__init__()
    
    def layout(self, params=None):
        return html.Div([
            self.querystring(params)(dcc.Input)(id=self.id("input1")),
            self.querystring(params, "min", "max")(dcc.Input)(id=self.id("input2")),
            self.querystring(params, "min", "max", "value")(dcc.Input)(id=self.id("input3"))
        ])
            

In [ ]:
t= T(name="0")
t.get_querystring_params()

[('input1-0', 'value'),
 ('input2-0', 'min'),
 ('input2-0', 'max'),
 ('input3-0', 'value'),
 ('input3-0', 'min'),
 ('input3-0', 'max')]

In [ ]:
t= T(name="0")
t.get_querystring_params()
input1_attributes = [qs[1] for qs in t.get_querystring_params() if qs[0]=="input1-0"]
input2_attributes = [qs[1] for qs in t.get_querystring_params() if qs[0]=="input2-0"]
input3_attributes = [qs[1] for qs in t.get_querystring_params() if qs[0]=="input3-0"]

assert input1_attributes == ['value'], "if no attributes explicitly listen, default to track 'value'"
assert not 'value' in input2_attributes, "value should not be in attributes if other attributes are explicitly listed"
assert input3_attributes[0] == 'value', "value should always be the first attributes in the list!"

#### `DashComponent` example: `ListComponent`

In [ ]:
class ListComponent(DashComponent):
    def __init__(self, list_factory, first_n=2, name=None):
        super().__init__()
        
    def layout(self, params=None):
        return html.Div([
            self.querystring(params)(dcc.Input)(
                id=self.id("input-first-n"), 
                type="number", 
                value=self.first_n,
                min=0,
                max=self.list_factory.list_length()),
            html.Div(id=self.id("output-div"), 
                     children=" ".join(self.list_factory.return_list(self.first_n))),
        ])
    
    def component_callbacks(self, app):
        @app.callback(
            self.Output("output-div", "children"),
            self.Input("input-first-n", "value")
        )
        def update_div(first_n):
            if first_n is not None:
                return " ".join(self.list_factory.return_list(first_n))
            raise PreventUpdate

In [ ]:
list_component = ListComponent(list_factory, name="1")

In [ ]:
list_component.get_querystring_params()

[('input-first-n-1', 'value')]

In [ ]:
assert list_component.list_factory.return_list() == ['this', 'is', 'a', 'dumb', 'example']
assert isinstance(list_component.layout(), html.Div)
assert list_component.get_querystring_params()[0][1] == 'value'

In [ ]:
print(list_component.to_yaml())

dash_component:
  class_name: ListComponent
  module: __main__
  params:
    list_factory:
      dash_figure_factory:
        class_name: ListFactory
        module: __main__
        params:
          list_input:
          - this
          - is
          - a
          - dumb
          - example
          filepath: file_factory.pkl
    first_n: 2
    name: '1'



Store and reload the component (with the figure factory automatically getting reloaded as well!):

In [ ]:
list_component.to_yaml("list_component.yaml")

In [ ]:
list_component2 = ListComponent.from_yaml("list_component.yaml")
assert isinstance(list_component2, ListComponent)

In [ ]:
list_component.dump()
list_component2 = ListComponent.from_yaml("list_component.yaml", try_pickles=True)
assert isinstance(list_component2, ListComponent)

Couldn't find file_factory.pkl! So loading from config instead...


In [ ]:
assert list_component2.list_factory.return_list() == list_component.list_factory.return_list()

In [ ]:
list_factory.to_config()['dash_figure_factory']['params']['filepath']

'file_factory.pkl'

See that `force_pickle=True` fails when the `DashFigureFactory`'s filepath is non-existent, but
that `try_pickles=True` simply reload the `DashFigureFactory` from config:

In [ ]:
list_factory._stored_params['filepath'] = "non_existing_file.pkl"
list_component = ListComponent(list_factory)
list_component.to_yaml("list_component.yaml")
try:
    list_component.from_yaml("list_component.yaml", force_pickles=True)
except Exception as e:
    assert isinstance(e, FileNotFoundError)
list_factory._stored_params['filepath'] = "file_factory.pkl"
list_component.to_yaml("list_component.yaml")

#### Composite `DashComponent` Example: `ListComposite`

A `ListComposite` if a combination of two `ListComponents`, both with different initial settings for `first_n`.

- the subcomponents get defined in the init with `self.list1 = ...` and `self.list2 = ...`
- the subcomponents get included in the layout by including `self.list1.layout()` and `self.list2.layout()`
- callbacks can be written that include elements from the ListComposite and the subcomponents
    - you need to add the `.name` from the subcomponents to properly identify the element id's
    - in this case the reset button resets the two inputs of the subcomponents:
        ```python
        @app.callback(
            Output("input-first-n-"+self.list1.name, "value"),
            Output("input-first-n-"+self.list2.name, "value"),
            Input("reset-button-"+self.name, "n_clicks")
        )
        ```
- additional callbacks should be defined under `_register_callbacks(self, app)`! (**note the underscore!**)
- callbacks of the subcomponents also get automatically registered, and included when `DashApp` calls
    `register_components(app)` (**note the lack of underscore here!**)

In [ ]:
class ListComposite(DashComponent):
    def __init__(self, list_factory, first_n1=2, first_n2=3, name=None):
        super().__init__()
        self.list1 = ListComponent(list_factory, first_n=first_n1, name="1")
        self.list2 = ListComponent(list_factory, first_n=first_n2, name="2")
        
    def layout(self, params=None):
        return html.Div([
            html.Button("Reset", id=self.id("reset-button")),
            self.querystring(params)(DashComponentTabs)(
                component=self, id="tabs", tabs=[self.list1, self.list2], params=params)
        ])
    
    def component_callbacks(self, app):
        @app.callback(
            self.list1.Output("input-first-n", "value"),
            self.list2.Output("input-first-n", "value"),
            self.Input("reset-button", "n_clicks")
        )
        def reset_inpus(n_clicks):
            if n_clicks:
                return self.first_n1, self.first_n2
            raise PreventUpdate

In [ ]:
list_composite = ListComposite(list_factory, name="main")

In [ ]:
list_composite.get_querystring_params()

[('input-first-n-1', 'value'), ('input-first-n-2', 'value')]

In [ ]:
list_composite.layout()

Div([Button(children='Reset', id='reset-button-main'), Tabs(children=[Tab(children=Div([Input(id='input-first-n-1', value=2, type='number', max=5, min=0), Div(children='this is', id='output-div-1')]), id='tabs-1', label='Dash', value='1'), Tab(children=Div([Input(id='input-first-n-2', value=3, type='number', max=5, min=0), Div(children='this is a', id='output-div-2')]), id='tabs-2', label='Dash', value='2')], id='tabs', value='1')])

In [ ]:
list_composite._tab_params

{'tabs': {'1': [('input-first-n-1', 'value')],
  '2': [('input-first-n-2', 'value')]}}

In [ ]:
print(list_composite._querystring_params)
print(list_composite.list1._querystring_params)
print(list_composite.list2._querystring_params)

[]
[('input-first-n-1', 'value')]
[('input-first-n-2', 'value')]


In [ ]:
list_composite.compute_querystring_params()
querystring_params = list_composite.get_querystring_params()
assert len(querystring_params) == 3, \
 "should be three querystring params, one from each subcomponent plus 'tabs'!"
querystring_params

[('tabs', 'value'), ('input-first-n-1', 'value'), ('input-first-n-2', 'value')]

In [ ]:
unreachable_querystring_params = list_composite.get_unreachable_querystring_params()
assert len(unreachable_querystring_params) == 0, \
    "should be no unreachable params in this component!"
unreachable_querystring_params

[]

In [ ]:
print(list_composite.to_yaml())

dash_component:
  class_name: ListComposite
  module: __main__
  params:
    list_factory:
      dash_figure_factory:
        class_name: ListFactory
        module: __main__
        params:
          list_input:
          - this
          - is
          - a
          - dumb
          - example
          filepath: file_factory.pkl
    first_n1: 2
    first_n2: 3
    name: main



subcomponents are automatically registerd to a `_components` list when calling `register_components()`.

(this gets called initially in `register_callbacks()`)

In [ ]:
list_composite.register_components()
list_composite._components
assert len(list_composite._components) == 2

When we build a dash app from this list_composite, we can check that all three callbacks (one for the composite, and one each for each subcomponent) have indeed been registered:

In [ ]:
app = dash.Dash()
app.layout = list_composite.layout()
list_composite.register_callbacks(app)
assert len(app.callback_map) == 3, \
    ("Should be three callbacks (one for the composite, and one each for each "
     "subcomponent) have indeed been registered)")

### Using `DashConnectors` between `DashComponents`:

You can define `DashConnectors` that connect the inputs and outputs of multiple `DashComponents`. 
You can do this because you have access to the `.name` property of each subcomponent.

`DashConnectors` never register callbacks from any of its subcomponents.

Using DashConnectors can help clean up your code, especially if you can re-use them.

In [ ]:
class ResetConnector(DashConnector):
    def __init__(self, list_composite, list_component1, list_component2):
        super().__init__()
        
    def component_callbacks(self, app):
        @app.callback(
            [self.list_component1.Output("input-first-n", "value"),
             self.list_component2.Output("input-first-n", "value")],
             self.list_composite.Input("reset-button", "n_clicks")
        )
        def reset_inpus(n_clicks):
            if n_clicks:
                return self.list_composite.first_n1, self.list_composite.first_n2
            raise PreventUpdate
            
class ListComposite(DashComponent):
    def __init__(self, list_factory, first_n1=2, first_n2=3, name=None):
        super().__init__()
        self.list1 = ListComponent(self.list_factory, first_n=first_n1, name="1")
        self.list2 = ListComponent(self.list_factory, first_n=first_n2, name="2")
        self.connector = ResetConnector(self, self.list1, self.list2)
        
    def layout(self, params=None):
        return html.Div([
            html.Button("Reset", id="reset-button-"+self.name),
            self.querystring(params)(DashComponentTabs)(
                component=self, id="tabs", tabs=[self.list1, self.list2], params=params)
        ])
        

In [ ]:
list_composite = ListComposite(list_factory)
print(list_composite.to_yaml())

dash_component:
  class_name: ListComposite
  module: __main__
  params:
    list_factory:
      dash_figure_factory:
        class_name: ListFactory
        module: __main__
        params:
          list_input:
          - this
          - is
          - a
          - dumb
          - example
          filepath: file_factory.pkl
    first_n1: 2
    first_n2: 3
    name: GmrYyUT7T8



In [ ]:
#export

def concat_docstring(source=None):
    "Decorator: `__doc__` from `source` to __doc__"
    def _f(f):
        if isinstance(f, types.FunctionType):
            from_f = f
        else:
            from_f = f.__init__
            
        if isinstance(source, types.FunctionType):
            source_f = source
        elif source.__init__.__doc__ is not None:
            source_f = source.__init__
        else:
            source_f = source
        from_f.__doc__ = (
            str(from_f.__doc__) + "\n\n\"" + 
            f"Docstring from {source.__name__}" +
            "\n\n" +
            str(source_f.__doc__))
        return f
    return _f

In [ ]:
#export
def parse_url_to_params(url):
    """
    Returns a dict that summarizes the state of the app at the time that the
    querystring url was generated. Lists are (somewhat hackily) detected 
    by the first char=='['), get evaluated using ast. Numbers are 
    appropriately cast as either int or float.
    
    Params:
        url (str): url to be parsed. The querystring parameters should
            come in pairs e.g.:?input-id=value&binput-id=1
    
    Returns:
        dict: dictionary with the component_id as key and a list 
            of (param, value) pairs (e.g. {'input-id': [('value', 1)]}
    """

    parse_result = urlparse(url)

    statedict = parse_qs(parse_result.query)
    if statedict:
        for key, values in statedict.items():
            if len(values) % 2 == 0: # even length means no value attr
                new_value = list(map(list, zip(values[0::2], values[1::2])))
                statedict[key] = new_value
            else: # uneven length means value attr is the first element
                new_value = list([['value', values[0]]])
                new_value.extend(list(map(list, zip(values[1::2], values[2::2]))))
                statedict[key] = new_value
                
            # go through every parsed value pv and check whether it is a list
            # or a number and cast appropriately:
            for pv in statedict[key]:
                # if it's a list
                if isinstance(pv[1], str) and pv[1][0]=='[':
                    pv[1] = ast.literal_eval(pv[1])

                #if it's a number
                if (isinstance(pv[1], str) and
                    pv[1].lstrip('-').replace('.','',1).isdigit()):

                    if pv[1].isdigit():
                        pv[1] = int(pv[1])
                    else:
                        pv[1] = float(pv[1])
    else: #return empty dict
        statedict = dict()
    return statedict

In [ ]:
#export
def parse_url_to_qs_and_vals(url):
    """
    Returns a dict that summarizes the state of the app at the time that the
    querystring url was generated. Lists are (somewhat hackily) detected 
    by the first char=='['), get evaluated using ast. Numbers are 
    appropriately cast as either int or float.
    
    Params:
        url (str): url to be parsed. The querystring parameters should
            come in pairs e.g.:?input-id=value&binput-id=1
    
    Returns:
        dict: dictionary with the component_id as key and a list 
            of (param, value) pairs (e.g. {'input-id': [('value', 1)]}
    """

    parse_result = urlparse(url)

    statedict = parse_qs(parse_result.query)
    qs_params = []
    values = []
    
    if statedict:
        for key, vals in statedict.items():
            if len(vals) % 2 == 0: # even length means no value attr
                qs_params.extend(list(map(tuple, zip([key]*int(0.5*len(vals)), vals[0::2]))))
                values.extend(vals[1::2])
            else: # uneven length means value attr is the first element
                qs_params.append(tuple([key, "value"]))
                values.append(vals[0])
                                 
                qs_params.extend(list(map(tuple, zip([key]*int(0.5*(len(vals)-1)), vals[1::2]))))
                values.extend(vals[2::2])

    for i in range(len(values)):
        if isinstance(values[i], str) and values[i][0]=='[':
            values[i] = ast.literal_eval(values[i])

        #if it's a number
        if (isinstance(values[i], str) and
            values[i].lstrip('-').replace('.','',1).isdigit()):

            if values[i].isdigit():
                values[i] = int(values[i])
            else:
                values[i] = float(values[i])

    return qs_params, values

In [ ]:
#export
def encode_querystring_params_to_url(querystring_params, values):
    """encodes a list of querystring_params and a list of values to 
    a url. 
    
    Params:
        querystring_params (list[tuples]): format e.g. 
            [('input-id', 'value'), ('input-id', 'type')]
        values (list): list of values to be encoded, e.g.
            [1, 'number']
            
    Returns:
        str: url of format ?input-id=value&binput-id=1&input-id=type&binput-id='number'     
    """
    statelist = [(id, tuple([val])) if attr=="value" else (id, (attr, val)) 
                     for (id, attr), val
                         in zip(querystring_params, values) if val is not None]
    params = urlencode(statelist,  doseq=True)
    return f'?{params}'

In [ ]:
#export
def update_url_with_new_params(old_url, qs_params, vals):
    old_qs_params, old_vals = parse_url_to_qs_and_vals(old_url)
    for qs_param, val in zip(qs_params, vals):
        if qs_param in old_qs_params:
            old_vals[old_qs_params.index(qs_param)] = val
        else:
            old_qs_params.append(qs_param)
            old_vals.append(val)
    return encode_querystring_params_to_url(old_qs_params, old_vals)
    

## DashApp

In order to run your `DashboardComponent` dashboard you can pass it to a `DashApp` and run it:

```python
DashApp(dashboard_component).run()
```

Args:
- `dashboard_component` (DashComponent): component to be run
- `port` (int): port to run the server
- `mode` ({'dash', 'external', 'inline', 'jupyterlab'}): type of dash server to start
- `querystrings` (bool): save state to url querystring and load from querystring
- `kwargs`: all kwargs will be passed down to dash.Dash. See below the docstring of dash.Dash


In [ ]:
#export

class DashApp(DashComponentBase):
    """Wrapper class for dash apps. Assigns layout and callbacks from 
    a DashComponent to a Dash app, and runs it.
    
    Can run both Dash and JupyterDash apps.
    
    """
    @concat_docstring(dash.Dash)
    def __init__(self, dashboard_component,  
                 port=8050, mode='dash', querystrings=False, bootstrap=False,
                 **kwargs):
        """
        
        Args:
            dashboard_component (DashComponent): component to be run
            port (int): port to run the server
            mode ({'dash', 'external', 'inline', 'jupyterlab'}): type of dash server to start
            querystrings (bool): save state to querystring and load from querystring
            bootstrap: include default bootstrap css
            kwargs: all kwargs will be passed down to dash.Dash. See below the docstring of dash.Dash
            
        Returns:
            DashApp: simply start .run() to start the dashboard
        """
        super().__init__(child_depth=2)
        self._stored_params['dashboard_component'] = dashboard_component.to_config()
        self.app = self._get_dash_app()
                
    def _get_dash_app(self):
        if self.bootstrap:
            bootstrap_theme = self.bootstrap if isinstance(self.bootstrap, str) else dbc.themes.BOOTSTRAP
            if 'external_stylesheets' not in self.kwargs:
                self.kwargs['external_stylesheets'] = [bootstrap_theme]
            else:
                self.kwargs['external_stylesheets'].append(bootstrap_theme)
                
        if self.querystrings:
            self.kwargs["suppress_callback_exceptions"] = True
        if self.mode == 'dash':
            app = dash.Dash(**self.kwargs)
        elif self.mode in {'inline', 'external', 'jupyterlab'}:
            app = jupyter_dash.JupyterDash(**self.kwargs)
            
        if not self.querystrings:
            app.layout = self.dashboard_component.layout()
            
        else:
            
            try:
                self.dashboard_component.compute_querystring_params()
                self.dashboard_component.layout(None)
            except Exception as e:
                print(e)
                raise ValueError("The layout method method of dashboard_component does not take "
                                 "a params parameter. Please rewrite as `def layout(self, params=None):` !")
            
            unreachable_params = self.dashboard_component.get_unreachable_querystring_params()
            if unreachable_params:
                print("Warning: The following elements will be tracked in the querystring, "
                      "but do not get passed as params to the (subcomponent) .layout(params) function, and so "
                      "will not be rebuilt when reloading from the url querystring! Please "
                      "make sure that you pass params down to the layout of all subcomponents! "
                      "e.g. def layout(self, params=None): return html.Div([self.sub_component.layout(params)]) \n\n",
                      unreachable_params)
            app.layout = html.Div([
                        dcc.Location(id='url', refresh=False),
                        html.Div(id='page-layout')
                    ])

            @app.callback(Output('page-layout', 'children'),
                  [Input('url', 'href')])
            def page_load(href):
                if not href:
                    return html.Div()
                params = parse_url_to_params(href)
                return self.dashboard_component.layout(params)
            
            @app.callback(Output('url', 'search'),
                          [Input(id, param) for (id, param) 
                                  in self.dashboard_component.get_querystring_params()],
                         [State('url', 'search')],
                         prevent_initial_call=True
                 )
            def update_url_state(*values):
                if hasattr(self.dashboard_component, "_tab_params"):
                    tab_params = self.dashboard_component._tab_params
                    qs_params = self.dashboard_component.get_querystring_params()
                    qs_keys, qs_attrs = zip(*qs_params)
                    
                    excluded_params = []
                    for tab in set(qs_keys) & set(tab_params.keys()):
                        tab_value = values[qs_params.index(tuple([tab, "value"]))]
                        for tab_name, tab_qs  in tab_params[tab].items():
                            if tab_name != tab_value:
                                excluded_params.extend(tab_qs)
                                
                    qs_vals = [(qs, v) for qs, v in zip(qs_params, values) if qs not in excluded_params]
                    q, v = zip(*qs_vals)
                    return encode_querystring_params_to_url(q, v)
                    
                old_url = values[-1]
                ctx = dash.callback_context
                params = [tuple(trigger['prop_id'].split('.')) for trigger in ctx.triggered]
                idxs = [self.dashboard_component.get_querystring_params().index(param) 
                            for param in params]
                vals = list(map(values.__getitem__, idxs))
                return update_url_with_new_params(old_url, params, vals)
                    
            
        self.dashboard_component.register_callbacks(app)

        app.title = self.dashboard_component.title
        return app
    
    def to_config(self):
        return dict(dash_app=dict(
            class_name=self.__class__.__name__, 
            module=self.__class__.__module__,
            params=self._stored_params))
    
    def flask_server(self):
        """returns flask server inside self.app, for building wsgi apps"""
        return self.app.server
    
    def run(self, port=None):
        """Run the dash app"""
        self.app.run_server(port=port if port is not None else self.port)
        

In [ ]:
show_doc(DashApp)
show_doc(DashApp.__init__)
show_doc(DashApp.flask_server)
show_doc(DashApp.run)

<h2 id="DashApp" class="doc_header"><code>class</code> <code>DashApp</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>DashApp</code>(**`dashboard_component`**, **`port`**=*`8050`*, **`mode`**=*`'dash'`*, **`querystrings`**=*`False`*, **`bootstrap`**=*`False`*, **\*\*`kwargs`**) :: [`DashComponentBase`](/dash_oop_components/core.html#DashComponentBase)

Wrapper class for dash apps. Assigns layout and callbacks from 
a DashComponent to a Dash app, and runs it.

Can run both Dash and JupyterDash apps.

<h4 id="DashApp.__init__" class="doc_header"><code>DashApp.__init__</code><a href="__main__.py#L10" class="source_link" style="float:right">[source]</a></h4>

> <code>DashApp.__init__</code>(**`dashboard_component`**, **`port`**=*`8050`*, **`mode`**=*`'dash'`*, **`querystrings`**=*`False`*, **`bootstrap`**=*`False`*, **\*\*`kwargs`**)

        
        Args:
            dashboard_component (DashComponent): component to be run
            port (int): port to run the server
            mode ({'dash', 'external', 'inline', 'jupyterlab'}): type of dash server to start
            querystrings (bool): save state to querystring and load from querystring
            bootstrap: include default bootstrap css
            kwargs: all kwargs will be passed down to dash.Dash. See below the docstring of dash.Dash
            
        Returns:
            DashApp: simply start .run() to start the dashboard
        

"Docstring from Dash

Dash is a framework for building analytical web applications.
    No JavaScript required.

    If a parameter can be set by an environment variable, that is listed as:
        env: ``DASH_****``
    Values provided here take precedence over environment variables.

    :param name: The name Flask should use for your app. Even if you provide
        your own ``server``, ``name`` will be used to help find assets.
        Typically ``__name__`` (the magic global var, not a string) is the
        best value to use. Default ``'__main__'``, env: ``DASH_APP_NAME``
    :type name: string

    :param server: Sets the Flask server for your app. There are three options:
        ``True`` (default): Dash will create a new server
        ``False``: The server will be added later via ``app.init_app(server)``
            where ``server`` is a ``flask.Flask`` instance.
        ``flask.Flask``: use this pre-existing Flask server.
    :type server: boolean or flask.Flask

    :param assets_folder: a path, relative to the current working directory,
        for extra files to be used in the browser. Default ``'assets'``.
        All .js and .css files will be loaded immediately unless excluded by
        ``assets_ignore``, and other files such as images will be served if
        requested.
    :type assets_folder: string

    :param assets_url_path: The local urls for assets will be:
        ``requests_pathname_prefix + assets_url_path + '/' + asset_path``
        where ``asset_path`` is the path to a file inside ``assets_folder``.
        Default ``'assets'``.
    :type asset_url_path: string

    :param assets_ignore: A regex, as a string to pass to ``re.compile``, for
        assets to omit from immediate loading. Ignored files will still be
        served if specifically requested. You cannot use this to prevent access
        to sensitive files.
    :type assets_ignore: string

    :param assets_external_path: an absolute URL from which to load assets.
        Use with ``serve_locally=False``. Dash can still find js and css to
        automatically load if you also keep local copies in your assets
        folder that Dash can index, but external serving can improve
        performance and reduce load on the Dash server.
        env: ``DASH_ASSETS_EXTERNAL_PATH``
    :type assets_external_path: string

    :param include_assets_files: Default ``True``, set to ``False`` to prevent
        immediate loading of any assets. Assets will still be served if
        specifically requested. You cannot use this to prevent access
        to sensitive files. env: ``DASH_INCLUDE_ASSETS_FILES``
    :type include_assets_files: boolean

    :param url_base_pathname: A local URL prefix to use app-wide.
        Default ``'/'``. Both `requests_pathname_prefix` and
        `routes_pathname_prefix` default to `url_base_pathname`.
        env: ``DASH_URL_BASE_PATHNAME``
    :type url_base_pathname: string

    :param requests_pathname_prefix: A local URL prefix for file requests.
        Defaults to `url_base_pathname`, and must end with
        `routes_pathname_prefix`. env: ``DASH_REQUESTS_PATHNAME_PREFIX``
    :type requests_pathname_prefix: string

    :param routes_pathname_prefix: A local URL prefix for JSON requests.
        Defaults to ``url_base_pathname``, and must start and end
        with ``'/'``. env: ``DASH_ROUTES_PATHNAME_PREFIX``
    :type routes_pathname_prefix: string

    :param serve_locally: If ``True`` (default), assets and dependencies
        (Dash and Component js and css) will be served from local URLs.
        If ``False`` we will use CDN links where available.
    :type serve_locally: boolean

    :param compress: Use gzip to compress files and data served by Flask.
        Default ``True``
    :type compress: boolean

    :param meta_tags: html <meta> tags to be added to the index page.
        Each dict should have the attributes and values for one tag, eg:
        ``{'name': 'description', 'content': 'My App'}``
    :type meta_tags: list of dicts

    :param index_string: Override the standard Dash index page.
        Must contain the correct insertion markers to interpolate various
        content into it depending on the app config and components used.
        See https://dash.plotly.com/external-resources for details.
    :type index_string: string

    :param external_scripts: Additional JS files to load with the page.
        Each entry can be a string (the URL) or a dict with ``src`` (the URL)
        and optionally other ``<script>`` tag attributes such as ``integrity``
        and ``crossorigin``.
    :type external_scripts: list of strings or dicts

    :param external_stylesheets: Additional CSS files to load with the page.
        Each entry can be a string (the URL) or a dict with ``href`` (the URL)
        and optionally other ``<link>`` tag attributes such as ``rel``,
        ``integrity`` and ``crossorigin``.
    :type external_stylesheets: list of strings or dicts

    :param suppress_callback_exceptions: Default ``False``: check callbacks to
        ensure referenced IDs exist and props are valid. Set to ``True``
        if your layout is dynamic, to bypass these checks.
        env: ``DASH_SUPPRESS_CALLBACK_EXCEPTIONS``
    :type suppress_callback_exceptions: boolean

    :param prevent_initial_callbacks: Default ``False``: Sets the default value
        of ``prevent_initial_call`` for all callbacks added to the app.
        Normally all callbacks are fired when the associated outputs are first
        added to the page. You can disable this for individual callbacks by
        setting ``prevent_initial_call`` in their definitions, or set it
        ``True`` here in which case you must explicitly set it ``False`` for
        those callbacks you wish to have an initial call. This setting has no
        effect on triggering callbacks when their inputs change later on.

    :param show_undo_redo: Default ``False``, set to ``True`` to enable undo
        and redo buttons for stepping through the history of the app state.
    :type show_undo_redo: boolean

    :param plugins: Extend Dash functionality by passing a list of objects
        with a ``plug`` method, taking a single argument: this app, which will
        be called after the Flask server is attached.
    :type plugins: list of objects

    :param title: Default ``Dash``. Configures the document.title
    (the text that appears in a browser tab).

    :param update_title: Default ``Updating...``. Configures the document.title
    (the text that appears in a browser tab) text when a callback is being run.
    Set to None or '' if you don't want the document.title to change or if you
    want to control the document.title through a separate component or
    clientside callback.
    

<h4 id="DashApp.flask_server" class="doc_header"><code>DashApp.flask_server</code><a href="__main__.py#L123" class="source_link" style="float:right">[source]</a></h4>

> <code>DashApp.flask_server</code>()

returns flask server inside self.app, for building wsgi apps

<h4 id="DashApp.run" class="doc_header"><code>DashApp.run</code><a href="__main__.py#L127" class="source_link" style="float:right">[source]</a></h4>

> <code>DashApp.run</code>(**`port`**=*`None`*)

Run the dash app

### Example use of `DashApp`

You can build and run dash app by simply passing a `DashComposite` to `DashApp` and then running it:

In [ ]:
db = DashApp(list_composite, mode='external', port=9000, querystrings=True, bootstrap=dbc.themes.FLATLY)

- You can set the port with `port=8051`
- You can run the dashboard inline in a notebook by pasing `mode='inline'`
    - or `mode='external'` or `mode='jupyterlab'`
    - default is `mode='dash'`
- Track parameters in the url querystring with querystrings=True
- Any additional parameters will be passed on the to `dash.Dash()` constructor

In [ ]:
#hide 
run_app = False

In [ ]:
if run_app:
    db.run()

You can also store and reload an entire dashboard:

In [ ]:
print(db.to_yaml())

dash_app:
  class_name: DashApp
  module: __main__
  params:
    dashboard_component:
      dash_component:
        class_name: ListComposite
        module: __main__
        params:
          list_factory:
            dash_figure_factory:
              class_name: ListFactory
              module: __main__
              params:
                list_input:
                - this
                - is
                - a
                - dumb
                - example
                filepath: file_factory.pkl
          first_n1: 2
          first_n2: 3
          name: GmrYyUT7T8
    port: 9000
    mode: external
    querystrings: true
    bootstrap: https://stackpath.bootstrapcdn.com/bootswatch/4.5.0/flatly/bootstrap.min.css
    kwargs:
      external_stylesheets:
      - https://stackpath.bootstrapcdn.com/bootswatch/4.5.0/flatly/bootstrap.min.css
      suppress_callback_exceptions: true



In [ ]:
db.to_yaml("dashboard.yaml")

In [ ]:
db2 = DashApp.from_yaml("dashboard.yaml", force_pickles=True)

In [ ]:
if run_app:
    db2.run()

In [ ]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_cli.ipynb.
Converted 02_Example.ipynb.
Converted 03_querystrings.ipynb.
Converted 04_GunicornDeployment.ipynb.
Converted index.ipynb.
